In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
import os
print(os.getcwd())

/home/demo/Desktop/Pi-Pupil-Detection/Pixel Error


In [3]:
cols = ['x', 'y']
# For one participant multiple data recording parts were used in LPW 
#1 - [1,4,9]   2 - [4,10,13]   3 - [16,19,21]
PATICIPANT_PART = 21
PATICIPANT_NUM = 3

# CANNY = 25
# BLUR = 9
# CANNY = 18
# BLUR = 21
# CANNY = 21
# BLUR = 13

path = rf"dataset/LPW_example/LPW_subject{PATICIPANT_NUM}"
detection_observed = pd.read_csv(path + rf'/Experiment_{PATICIPANT_PART}_canny_{CANNY}_blur_{BLUR}.csv', delimiter='\t')

detection_actual = pd.read_csv(path + rf'/{PATICIPANT_PART}.txt', delim_whitespace= True, header=None)
detection_actual.columns = cols
detection_actual[cols] = detection_actual[cols].astype('float')
detection_observed[cols] = detection_observed[cols].astype('float')

/tmp/ipykernel_3461/1359622930.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  detection_actual = pd.read_csv(path + rf'/{PATICIPANT_PART}.txt', delim_whitespace= True, header=None)


In [4]:
glob.glob(path+"\*avi")
#file_name = Path(path).stem

[]

In [5]:
detection_actual

,x,y
0,453.00,186.50
1,453.83,186.33
2,454.67,186.17
3,455.50,186.00
4,455.67,186.17
...,...,...
1995,364.00,155.50
1996,363.83,156.00
1997,363.67,156.50
1998,363.50,157.00


In [6]:
#detection_observed

In [7]:
observed_num = len(detection_observed)

In [8]:
misidentified = len(detection_observed[(detection_observed.extra != '[]') & (detection_observed.x != 0)])
misidentified

1298

In [9]:
notidentified = len(detection_observed[(detection_observed.x == 0)])
notidentified

23

In [10]:
identified = ((observed_num - misidentified - notidentified) * 100) / observed_num
identified

33.95

In [11]:
detection_observed[cols] - detection_actual[cols] 

,x,y
0,-36.078407,263.174342
1,-37.227859,263.355512
2,-37.661409,263.525720
3,-38.696200,263.917411
4,-38.498916,263.457714
...,...,...
1995,51.575464,293.176972
1996,52.107062,292.777565
1997,51.860982,292.451592
1998,52.176537,291.770940


In [12]:
# Error calculation (MSE)

evaluation = []

for data_o, data_a in zip(detection_observed.values, detection_actual.values):
    #print(np.linalg.norm([data_o[0] , data_a[0]], 1))
    ecu_dist = np.sqrt(np.square(data_o[0] - data_a[0]) + np.square(data_o[1] - data_a[1]))
    evaluation.append(ecu_dist)
    #np.linalg.norm([data_o[0], data_o[1]], [data_a[0], data_a[1]], axis=1)


In [13]:
filter_evaluation =  list(filter(lambda value: value < 100, evaluation))

In [14]:
#max(filter_evaluation)

In [15]:
error_evaluation_5pix =  list(filter(lambda value: value < 5, evaluation))
len(error_evaluation_5pix)

13

In [16]:
tolerated_detection_5pix = (len(error_evaluation_5pix) * 100) / observed_num
tolerated_detection_5pix

0.65

In [17]:
error_evaluation_10pix =  list(filter(lambda value: value < 10, evaluation))
len(error_evaluation_10pix)

16

In [18]:
tolerated_detection_10pix = (len(error_evaluation_10pix) * 100) / observed_num
tolerated_detection_10pix

0.8

In [19]:
# detection_rate_result = pd.DataFrame(columns = ['PATICIPANT_NUM', 'PATICIPANT_PART', 'CANNY_VALUE', 'BLUR_VALUE', 'DETECTION_RATE_5PIX', 'DETECTION_RATE_10PIX', 'FRAME_SIZE'])
detection_rate_result = pd.read_csv(path + rf'/detect_participant{PATICIPANT_NUM}.csv', delimiter=',')
detection_rate_result.loc[len(detection_rate_result.index)] = [PATICIPANT_NUM, PATICIPANT_PART, CANNY, BLUR,  tolerated_detection_5pix, tolerated_detection_10pix, '480x480']
detection_rate_result


,PATICIPANT_NUM,PATICIPANT_PART,CANNY_VALUE,BLUR_VALUE,DETECTION_RATE_5PIX,DETECTION_RATE_10PIX,FRAME_SIZE
0,3,16,25,9,0.00,0.00,480x480
1,3,16,18,21,0.00,0.00,480x480
2,3,16,21,13,0.00,0.05,480x480
3,3,19,25,9,0.00,0.00,480x480
4,3,19,18,21,0.00,0.00,480x480
5,3,19,21,13,0.00,0.00,480x480
6,3,21,25,9,1.50,1.95,480x480
7,3,21,18,21,0.10,0.15,480x480
8,3,21,21,13,0.65,0.80,480x480


In [20]:
detection_rate_result.to_csv(path + f"/detect_participant{PATICIPANT_NUM}.csv",
                     sep=',', index=False)

In [21]:
print(f"mean - {np.mean(filter_evaluation)}, median - {np.median(filter_evaluation)}, STD - {np.std(filter_evaluation)}")

mean - 7.491451113111733, median - 2.064580879970084, STD - 18.46802952719198


In [22]:
num_not_evaluated = len(evaluation) - len(filter_evaluation)
#max(filter_evaluation)

In [23]:
#for data in filter_evaluation:
#    print(data)
df_experiment = pd.DataFrame(filter_evaluation, columns=["Error_Euclidean"])

df_experiment.to_csv(path + f"/participant{PATICIPANT_NUM}_part_{PATICIPANT_PART}_error__canny_{CANNY}_blur_{BLUR}.csv",
                     sep=',', index=False)